<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150 align="right">
<img src=images/Acme.jpeg width=100 align="left">

<a id =1></a>
## Encryption as a Service / Cryptography as a Service / Transit Secrets Engine


## Business Values:
 * Enables security teams to fortify data during transit and at rest
 * Relieve the burden of proper encryption/decryption from application developers
 * Sign adn Verify data; generate hashes and HMACs of data and act as a source of random bytes
 * If an intrusion occurs, your data is encrypted with **AES 256-bit GCM** encryption
 * A centrally managed resource that can encrypt any data for the organization at scale
 * High performance - Vault will encrypt at higher rates (20K+ requests/sec) than the cloud providers solutions.
 
<img src=images/EncryptionAsaService.png width=900>
Transit Secret Engine supports a number of key types and some support signing and signature verification.

| Key Type |  Description |
|----------|--------------|
| aes128-gcm96 | AES-GCM with a 128-bit AES key and a 96-bit nonce |
| aes256-gcm96 | AES-GCM with a 256-bit AES key and a 96-bit nonce (**default**) |
| chachca20-poly1305 | ChaCha20-Poly1305 with a 256-bit key |
| ed25519 | ED 25519 |
|ecdsa-p256 | ECDSA using the P-256 elliptic curve |
|ecdsa-p256 | ECDSA using the P-384 elliptic curve |
|ecdsa-p521 | ECDSA using the P-521 elliptic curve |
| rsa-2048 | 2048-bit RSA key |
| rsa-3072 | 3072-bit RSA key |
| rsa-4096 | 4096-bit RSA key 

### Vault Setup

### Prerequisites
* Running Vault Enterprise instance (in dev mode) 


In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_SKIP_VERIFY=true

In [ ]:
vault login root

---
### Enable the Transit Secret Engine


In [ ]:
vault secrets enable transit


### Create an Encryption Key Ring

In [ ]:
vault write -f  transit/keys/orders #type=rsa-2048

In [ ]:
vault read transit/keys/orders

In [ ]:
vault write -f transit/keys/orders/rotate

In [ ]:
vault  read transit/export/encryption-key/orders

### Update key configurations
See a list of configurations using `vault read transit/keys/<keyname>` above

In [ ]:
vault write transit/keys/orders/config deletion_allowed=true exportable=true

In [ ]:
vault delete transit/keys/orders

---
### Encrypt Secrets

##### Note: Plaintext must be **base64-encoded** before it can be encrypted.

In [ ]:
PLAIN="This is my demo secret"
echo "Plaintext: "$PLAIN

In [ ]:
PLAINTEXT64=$(base64 <<< "$PLAIN")
echo "Plaintext in base64: " $PLAINTEXT64

In [ ]:
vault write transit/encrypt/orders  plaintext=$PLAINTEXT64  #key_version=0

**Note:** Vault does **NOT** store any of this data.  The output you received is ciphertext.  You can store this ciphertext at the desired location (e.g. mysql database) or pass it to your application.

We'll copy the value of ciphertext and refer to it later as "\<ciphertext\>"
    
Here's another way to encrypt using API (you can always use -output-curl-string)


In [ ]:
CIPHERTEXT=$(curl -s --header "X-Vault-Token: $(vault print token)" \
       --request POST \
       --data '{"plaintext": "'"${PLAINTEXT64}"'"}' \
       http://127.0.0.1:8200/v1/transit/encrypt/orders | jq -r '.data.ciphertext')

echo "Ciphertext: ${CIPHERTEXT}"

---
### Decrypt Ciphertext

In [ ]:
vault write transit/decrypt/orders ciphertext="vault:v1:2zV2ryzMkhjhhD743eRfOfuM4ik+pxn2K4GR+yQeJM+2jByBT6/YFWwkPN/6chdoSOBG"

\
Take the above output `plaintext` and execute a command
```
    base64 --decode <<< "<plaintext>"
```

In [ ]:
base64 --decode <<< "VGhpcyBpcyBteSBkZW1vIHNlY3JldAo="

&nbsp;

In [ ]:
DECIPHER=$(curl -s --header "X-Vault-Token: $(vault print token)" \
     --request POST \
     --data '{"ciphertext": "'"${CIPHERTEXT}"'"}' \
    http://127.0.0.1:8200/v1/transit/decrypt/orders | jq -r '.data.plaintext')


echo "Plaintext in Base 64 : ${DECIPHER}\n"
echo "Plaintext : "
echo "${DECIPHER}" | base64 --decode

---
### Rotate the Encryption key

One of the benefits of using the Vault transit secrets engine is its ability to easily rotate encryption keys. Keys can be rotated manually by a human or by an automated process.

In [ ]:
vault write -f transit/keys/orders/rotate

In [ ]:
vault write transit/encrypt/orders plaintext=$(base64 <<< "$PLAIN")

In [ ]:
## Encrypt using the http api 
CIPHERTEXT2=$(curl -s --header "X-Vault-Token: $(vault print token)" \
       --request POST \
       --data '{"plaintext": "'"${PLAINTEXT64}"'"}' \
       http://127.0.0.1:8200/v1/transit/encrypt/orders | jq -r '.data.ciphertext')

echo "Ciphertext after Rotation: ${CIPHERTEXT2}\n"
echo "Cipherext before Rotation: ${CIPHERTEXT}"

##### Notice that the first ciphertext starts with " vault:v1: ". After rotating the encryption key, the ciphertext starts with " vault:v2: ". This indicates that the data gets encrypted using the latest version of the key after the rotation

&nbsp;
### Rewrap the older version of ciphertext


In [ ]:
REWRAP_CIPHERTEXT=$(curl -s --header "X-Vault-Token: $(vault print token)" \
   --request POST \
   --data '{"ciphertext": "'"${CIPHERTEXT}"'" }' \
   http://127.0.0.1:8200/v1/transit/rewrap/orders | jq -r '.data.ciphertext')
   

echo "Original Ciphertext: ${CIPHERTEXT}\n"
echo "Rewrapped Ciphertext: ${REWRAP_CIPHERTEXT}\n"


#### Note that the rewrapped ciphertext has a **'v2'** in it. 

### Update Key Configuration

A Vault admin can update the encryption key configuration to specify the minimum version of ciphertext allowed to be decrypted,  
the minimum version of the key that can be used to encrypt the plaintext, and if the key is allowed to be deleted, etc. for _**increased security**_.

### Rotate keys a few times (more versions)

In [ ]:
export VAULT_TOKEN=root
vault write -f transit/keys/orders/rotate

### Read the order key information back

In [ ]:
vault read transit/keys/orders

##### The current version of the key is 7.  There is no restriction about the minimum key version and any of the key versions can decrypt the data (min_decyption_version)

### Set a Minimum Decryption Version

In [ ]:
vault write transit/keys/orders/config min_decryption_version=5

### Read the new Key Configuration

In [ ]:
vault read transit/keys/orders

In [ ]:
vault write transit/keys/orders/config min_decryption_version=2

&nbsp;
##### What happen to those ciphertext that were encrypted with v1, v2, or v3 of the key?
Answer:  Vault would refuse to decrypt the data as the key used is less than the minimum key version allowed.

In [ ]:
vault read transit/keys/orders

---
## Data Key
  
**Use case:** you have a large (say 2GB) base64 blob that needs to be encrypted.  You dont want to send large data over the network and receive encrypted data back.  Generate a data key to encrypt data **locally**.  Likewise, use the same data key to decrypt data locally when needed. 

In [ ]:
vault write -f transit/datakey/plaintext/orders

**Note:**  Use the plaintext to ecnrypt your blob.  Store the ciphertext of your data key whereever you want, incl key/value secrets engine.

### Use openssl to encrypt and decrypt data using the generated data key

In [ ]:
echo "b2R5jAUa92id30O3wmjZj8DVoWpdSklViYKN1pW3RNY=" > /tmp/key.bin

In [ ]:
rm /tmp/hello.enc /tmp/hello.in /tmp/hello.out; echo "hello world" > /tmp/hello.in;

In [ ]:
openssl enc -aes-256-cbc -salt -in /tmp/hello.in -out /tmp/hello.enc -pass file:/tmp/key.bin

In [ ]:
openssl enc -d -aes-256-cbc -salt -in /tmp/hello.enc -out /tmp/hello.out  -pass file:/tmp/key.bin

In [ ]:
cat /tmp/hello.in 
#cat /tmp/hello.enc 
cat /tmp/hello.out

&nbsp;   
**Misc: Generate Random bytes of certain length**

In [ ]:
curl -s --header "X-Vault-Token: $(vault print token)" \
   --request POST \
   http://127.0.0.1:8200/v1/transit/random/10 |jq

In [ ]:
echo "aGVsbG8gd29ybGQ=" |base64 --decode

In [ ]:
curl -s --header "X-Vault-Token: $(vault print token)" \
   --request POST \
   http://127.0.0.1:8200/v1/transit/cache-config |jq

vault write transit/encrypt/orders plaintext=$PLAINTEXT64#### Create another Encryption Key Ring

---
### Sign and Validate data
Consider a time where you want to ensure the data came from a trusted source.  You don't care who can read the data, but you care about the source of the data.   In such case, you use data signing instead of encryption.

see https://www.vaultproject.io/api-docs/secret/transit#create-key


In [ ]:
vault write transit/keys/newsletter type="rsa-4096"

In [ ]:
vault write transit/sign/newsletter input=$(base64 <<< "$PLAIN")

In [ ]:
# replace <signature> with the real data
vault write transit/verify/newsletter input=$(base64 <<< "$PLAIN") \
   signature="vault:v1:T1k9d5jbWYMm+GmBEYfPVWvpPSMMrzIFmU39RTrUe3meVq/LCee1lzcWCGwFOIofbA93vKJce/1Zx3lFAV+mN7Gp7hFbap1NfK964q/cbh759CEbZIE4sNt2t3ke1cXpeFJZ73KQgBzr/teZ3zrVNGJrtgAVSbo2mLoxkdjrkhymJFlqsyK2wxeLn+hCbahsI/QOYCBnlkUo1HsoMQx9DVjU2corOMDYnhHcniNsW5WGqIGive6Wet3d9lbKrKiP5EMn7MYhUrZ/xOcLErU4Th5YQWjSYj+tkpSWp6i2yR0Rgaw+6TLf5eieqHRYXvL9qJeWWt2ao4f3nUm6+40nQDPQVsF6hk0ihjb4dlYBKc5gO/mkqfCGftcoayJqujQ62535sLbCGmZObcb27CrPDi7GzBcdySh0z48tCQlerMPFk74lhnj2Yh4r+TonQXdCy6lr4PR+8hfv1edntImGQdcOmtzhwo/NnCeNKLO68XeoCYL3XJm1LKQIb56e5d7CwoEb1bfj349F4ZGNO2xsIHxqrUt5DqRg0g29bxCnnTI3/OcDEJJOZtop0Bd6srqLmYcG3LRacO9watORbC50s9XG6wkuQMJ8EP2kDoN2Bh+vFF2+BKQsX+JOc/wS1eTDEXU/UtN+Nw6aP8jSQjNbO7+ASJjabecOemSyOw+cED0="

---
### Encrypt data via web UI

Open a browser and visit http://127.0.0.1:8200


In [ ]:
vault delete transit/keys/newsletter

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">